In [3]:
!pip install gradio
!pip install gradio-client
!pip install -q gradio
!pip install transformers
!pip install gradio --upgrade
!pip install httpx==0.25.0


In [4]:
import gradio as gr
from transformers import pipeline
import numpy as np
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
def transcribe(audio):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))

    return transcriber({"sampling_rate": sr, "raw": y})["text"]

In [6]:
languages = ['hi', 'en']

def audio_analysis(audio):

  text = transcribe(audio)

  sentiment_polarity,sentiment_scores,max_emotion_label,max_sentiment,profanity,summary_text=analysis(text)

  return text,sentiment_polarity,sentiment_scores,max_emotion_label,max_sentiment,profanity,summary_text

### **Text Analysis**

In [7]:
!pip install tensorflow-text
!pip install tf-models-official
!pip install textblob
!pip install nltk
!pip install transformers
!pip install NRCEmotionLexicon
!pip install better_profanity

ERROR: Could not find a version that satisfies the requirement NRCEmotionLexicon (from versions: none)
ERROR: No matching distribution found for NRCEmotionLexicon


In [8]:
import tensorflow as tf
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import pipeline
from better_profanity import profanity
from transformers import BartForConditionalGeneration, BartTokenizer

model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)
nltk.download('vader_lexicon')
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
model_path = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a mode

In [9]:
def analysis(text):
  text=text[:512]
  #########TEXTBLOB#############
  analysis = TextBlob(text)
  sentiment_polarity = analysis.sentiment.polarity
  #########VADER#################
  analyzer = SentimentIntensityAnalyzer()
  sentiment_scores = analyzer.polarity_scores(text)
  #########Emotion #############
  output=classifier(text)
  max_emotion_data = max(output[0], key=lambda x: x['score'])
  max_emotion_label = max_emotion_data['label']
  #########Sentiment_BERT############
  sentiment=sentiment_task(text)
  max_sentiment=sentiment[0]["label"]
  #########Curse_Check################
  if(profanity.contains_profanity(text)):
    prof="The text contains profane words"
  else:
    prof="No text does not contain profane words"
  ##########Summarization####################
  input_ids = tokenizer(text, return_tensors="pt", max_length=2048, truncation=True)
  summary_ids = model.generate(input_ids["input_ids"], max_length=50, min_length=1, num_beams=4, length_penalty=2.0, early_stopping=True)
  summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  return sentiment_polarity*100,sentiment_scores['compound']*100,max_emotion_label,max_sentiment,prof,summary_text

In [10]:
demo = gr.Interface(
    fn=audio_analysis,
    inputs=gr.Audio(sources="microphone"),
    outputs=["text", "text", "text", "text", "text", "text", "text"],
    title="Shruti Deep Converse",
    description="An audio-based chatbot, equipped with both speech recognition and generation capabilities, actively listens and communicates through spoken language. This empathetic AI provides real-time support to those facing emotional distress, ensuring privacy and guiding users to mental health resources while engaging in natural voice conversations."
)

# Show the Gradio interface in the Colab notebook
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2cedd843fe5345bffb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
